##  FOUNDATIONS OF COMPUTER SCIENCE PROJECT

### Dario Della Mura - 793751

Carico le Librerie necessarie per il progetto.

In [1]:
import pandas as pd
import numpy as np
import datetime
from IPython.display import display

#### 1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

Carico il dataset loan_lenders:

In [2]:
loans_lenders= pd.read_csv('D:/UNIVERSITÀ/PROGETTI/FOCS/DATASET/loans_lenders.csv')
#loans_lenders.head()

Il mio obbiettivo è quello di normalizzare il dataset "lenders" ovvero renderlo atomico: ogni cella deve contenere un solo elemento. Per poter normalizzare il datatset, utilizzo la funzione "explode" che consente di dividere gli elementi all'interno di una cella in più celle. Attraverso str.split identifico il delimitatore, nel mio caso la " , ", ovvero qul carattere dopo il quale applicare la "explode" :

In [3]:
loans_lenders_nor = loans_lenders.assign(lenders=loans_lenders['lenders'].str.split(',')).explode('lenders')
loans_lenders_nor.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


Non potendo controllare tutto il dataset, non posso essere sicuro che ogni singola osservazione contenuta nella  colonna "lenders" sia scritta allo stesso modo (in termini di spazio prima e dopo al singolo lender). Per tanto applico la funzione str.split per eliminare eventuali spazi prima e dopo al nome del singolo lender :

In [4]:
loans_lenders_nor["lenders"] = loans_lenders_nor["lenders"].str.strip()
#loans_lenders_nor.head()

#### 2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

Carico il datatset "loans". Per un problema di memoria non vengono visualizzate tutte le colonne quindi, per una corretta esplorazione del dataset, utilizzo la funzione "pd.set_option('display.max_columns', None)" in modo da forzare la visualizzazione di tutte le colonne all'interno detaset. Dopo di che imposto le colonne "disburse_time" e "planned_expiration_time" come colonne contenenti delle date :

In [5]:
pd.set_option('display.max_columns', None)
loans = pd.read_csv('D:/UNIVERSITÀ/PROGETTI/FOCS/DATASET/loans.csv')

loans.disburse_time = pd.to_datetime(loans.disburse_time)
loans.planned_expiration_time = pd.to_datetime(loans.planned_expiration_time)


Calcolo ora la duration di ogni prestito come differenza tra "planned_expiration_time" e "disburse_time" (scadenza prevista meno giorno di erogazione del prestito). Decido di tener conto solo del giorno e non di ore e fuso orario:

In [6]:
loans['planned_expiration_time'] = loans['planned_expiration_time'].dt.tz_localize(None)

loans['disburse_time'] = loans['disburse_time'].dt.tz_localize(None)

loans['planned_expiration_time'] = loans['planned_expiration_time'].dt.normalize()

loans['disburse_time'] = loans['disburse_time'].dt.normalize()

loans['duration']=  (loans['planned_expiration_time'] - loans['disburse_time'])
print(loans[['loan_id', 'loan_amount', 'disburse_time', 'planned_expiration_time', 'duration']].head(3))

   loan_id  loan_amount disburse_time planned_expiration_time duration
0   657307        125.0    2013-12-22              2014-02-14  54 days
1   657259        400.0    2013-12-20              2014-03-26  96 days
2   658010        400.0    2014-01-09              2014-02-15  37 days


#### 3) Find the lenders that have funded at least twice :

Utilizzo il dataset loans_lenders normalizzato e applico una groupby in modo da ragruppare per singoli lenders e utilizzare il count per contare quanti hanno effettuato almeno 2 prestiti: 

In [7]:
lenders_least_2_loans = pd.DataFrame(loans_lenders_nor.groupby('lenders').count()['loan_id']).reset_index()[['lenders','loan_id']].query('loan_id > 1').rename(columns={"loan_id" : "count_loans"})
lenders_least_2_loans.head(3)

,lenders,count_loans
0,000,40
1,00000,39
2,0002,70


#### 4) For each country, compute how many loans have involved that country as borrowers.

Decido di pulire il dataset loans in modo tale da utilizzare solo le osservazioni non nulle in "duration" e che abbiamo una duration positiva (in quanto una duration negativa avrebbe poco senso) : 

In [8]:
loans_no_null =  loans[loans["duration"].notnull()==True]
loans_cleaned = loans_no_null[loans_no_null["planned_expiration_time"] > loans_no_null["disburse_time"]]

Ora applico una groupby su "country_name" in modo tale da raggruppare i singoli paesi per poi effettuare una count che mi dica quanti prestiti sono stati erogati al singolo paese :

In [9]:
loans_for_country = loans_cleaned.groupby("country_name").count().reset_index()[["country_name", "loan_id"]].rename(columns={"loan_id" : "count_loans"})
loans_for_country.head(3)

,country_name,count_loans
0,Afghanistan,1
1,Albania,3075
2,Armenia,12809


#### 5) For each country, compute the overall amount of money borrowed.

Applico una groupby su "country_name" per poter raggruppare i singoli paesi e poi applicare una sum sull'attributo "loan_amount" (importo del prestito) in modo da poter capire quanto denaro è stato prestato ad ogni singolo paese :

In [10]:
tot_borrowed_from_each_country = loans_cleaned.groupby('country_name')['loan_amount'].sum().reset_index().rename(columns={"loan_amount" : "tot_borrowed"})
tot_borrowed_from_each_country.head(3)

,country_name,tot_borrowed
0,Afghanistan,6000.0
1,Albania,4307350.0
2,Armenia,20579275.0


#### 6) Like the previous point, but expressed as a percentage of the overall amount lent.

Calcolo l'importo totale dei prestiti erogati :

In [11]:
tot_loans = loans_cleaned['loan_amount'].sum()
#tot_loans

Ora divido l'ammontare prestato al singolo paese per l'importo totale dei prestiti erogati "tot_loans":

In [12]:
tot_borrowed_from_each_country_perc = loans_cleaned.groupby('country_name')['loan_amount'].sum().reset_index().rename(columns={"loan_amount" : "tot_borrowed"})
tot_borrowed_from_each_country_perc['tot_borrowed_%'] = (tot_borrowed_from_each_country_perc['tot_borrowed'] /  tot_loans)*100
tot_borrowed_from_each_country_perc.head(3)

,country_name,tot_borrowed,tot_borrowed_%
0,Afghanistan,6000.0,0.000692
1,Albania,4307350.0,0.497080
2,Armenia,20579275.0,2.374905


Creo un dataset unico in modo che contenga tutte le informazioni sui presti erogati ad ogni singolo paese (Numero di prestiti erogati, importo totale e percentuale sul totale) :

In [13]:
country_summary = pd.concat([loans_for_country, tot_borrowed_from_each_country, tot_borrowed_from_each_country_perc], axis=1)
country_summary = country_summary.loc[:,~country_summary.columns.duplicated()]
country_summary.head(3)

,country_name,count_loans,tot_borrowed,tot_borrowed_%
0,Afghanistan,1,6000.0,0.000692
1,Albania,3075,4307350.0,0.497080
2,Armenia,12809,20579275.0,2.374905


#### 7) Like the three previous points, but split for each year (with respect to disburse time).

Imposto come indice "disburse_time" :

In [14]:
loans_cleaned.index = pd.DatetimeIndex(loans_cleaned.disburse_time)

Effettuo un raggruppamento sia per "country_name" sia per anno, attraverso il comando pd.Grouper (indicando come parametri l'attributo su cui effettuare il raggruppamnto, "disburse_time", e il parametro "Y", ovvero anni) :

In [15]:
loans_for_year_for_each_country = loans_cleaned.groupby(["country_name", pd.Grouper(key= 'disburse_time', freq="Y")])['loan_id'].count().reset_index()[["country_name", "disburse_time","loan_id"]].rename(columns={"loan_id" : "count_loans_for_year", "disburse_time": "year"})
loans_for_year_for_each_country['year'] = loans_for_year_for_each_country['year'].dt.year

loans_for_year_for_each_country.head(3)

,country_name,year,count_loans_for_year
0,Afghanistan,2015,1
1,Albania,2012,332
2,Albania,2013,507


In [16]:
overall_borrowed_for_year = loans_cleaned.groupby(["country_name", pd.Grouper(key= 'disburse_time', freq="Y")])['loan_amount'].sum().reset_index()[["country_name", "disburse_time","loan_amount"]].rename(columns={"loan_amount" : "tot_borrowed_for_year", "disburse_time": "year"})
overall_borrowed_for_year.year = overall_borrowed_for_year.year.dt.year
overall_borrowed_for_year.head(3)

,country_name,year,tot_borrowed_for_year
0,Afghanistan,2015,6000.0
1,Albania,2012,413000.0
2,Albania,2013,786950.0


In [17]:
overall_loan_borrowed_for_year_perc =loans_cleaned.groupby(["country_name", pd.Grouper(key= 'disburse_time', freq="Y")])['loan_amount'].sum().reset_index().rename(columns={"disburse_time" : "year"})
overall_loan_borrowed_for_year_perc['loan_amount'] = (overall_loan_borrowed_for_year_perc['loan_amount']/tot_loans)*100
overall_loan_borrowed_for_year_perc.rename(columns={"loan_amount" : "tot_loan_borrowed_for_year_perc"})
overall_loan_borrowed_for_year_perc.year = overall_loan_borrowed_for_year_perc.year.dt.year
overall_loan_borrowed_for_year_perc.head(3)

,country_name,year,loan_amount
0,Afghanistan,2015,0.000692
1,Albania,2012,0.047661
2,Albania,2013,0.090816


Creo un unico dataset in modo tale che contenga tutte le informazioni sui prestiti erogati ai singoli paesi suddivise per anni :

In [18]:
country_summary_year = pd.concat([loans_for_year_for_each_country, overall_borrowed_for_year, overall_loan_borrowed_for_year_perc], axis=1)
country_summary_year = country_summary_year.loc[:,~country_summary_year.columns.duplicated()]
country_summary_year.head(3)

,country_name,year,count_loans_for_year,tot_borrowed_for_year,loan_amount
0,Afghanistan,2015,1,6000.0,0.000692
1,Albania,2012,332,413000.0,0.047661
2,Albania,2013,507,786950.0,0.090816


#### 8) For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


Creo un dataset che contenga per ogni prestito, "loan_id", il numero di finanziatori, "lenders" :

In [19]:
lenders_count= loans_lenders_nor.groupby("loan_id").count().reset_index().rename(columns={"lenders" : "lenders_count"})
lenders_count.head(3)

,loan_id,lenders_count
0,84,3
1,85,2
2,86,3


Unisco il daset appena creato, "lenders_count", con il dataset "loans_cleaned" in modo da visualizzare, per ogni prestito, il numero di finanziatori e il suo importo :

In [20]:
lenders_count_summary = pd.merge(lenders_count, loans_cleaned, on="loan_id")[["loan_id", "lenders_count", "loan_amount"]]
lenders_count_summary.head(3)

,loan_id,lenders_count,loan_amount
0,358219,74,2300.0
1,358926,60,1925.0
2,359566,42,1400.0


Per ogni prestito calcolo l'importo prestato da un singolo finanziatore (si suppono che, in caso di pià finanziatori per un singolo prestito, ogni finanziatore abbia contribuito allo stesso modo) :

In [21]:
lenders_count_summary['amount_for_lender'] = lenders_count_summary['loan_amount'] / lenders_count_summary['lenders_count']
lenders_count_summary.head(3)

,loan_id,lenders_count,loan_amount,amount_for_lender
0,358219,74,2300.0,31.081081
1,358926,60,1925.0,32.083333
2,359566,42,1400.0,33.333333


Effettuo una merge tra "loans_lenders_nor", che contiene i nomi dei finanziatori per ogni prestito da loro effettuato, e "lenders_count_summary" che contiene le informazioni sui singoli prestiti. In questo modo calcolo, per ogni prestito, l'ammontare prestato dal singolo finanziatore :

In [22]:
loans_for_lender = pd.merge(loans_lenders_nor, lenders_count_summary, on="loan_id", how="left")
loans_for_lender.head(3)

,loan_id,lenders,lenders_count,loan_amount,amount_for_lender
0,483693,muc888,40.0,1225.0,30.625
1,483693,sam4326,40.0,1225.0,30.625
2,483693,camaran3922,40.0,1225.0,30.625


Ora posso calcolare l'importo totale prestato da ogni finanziatore. Effettuo una groupby per finanziatore, "lenders", e applico la funzione sum così da poter individuare l'importo totale prestato da ogni singolo finanziatore :

In [23]:
tot_lent_by_lender = loans_for_lender.groupby("lenders")["amount_for_lender"].sum().to_frame().reset_index()
tot_lent_by_lender.head(3)

,lenders,amount_for_lender
0,000,1375.029391
1,00000,0.000000
2,0002,677.081981


#### 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Carico il dataset "lenders" : 

In [24]:
lenders= pd.read_csv('D:/UNIVERSITÀ/PROGETTI/FOCS/DATASET/lenders.csv')
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


Il dataset "lenders" contiene molti valori mancanti nell'attributo "country_code", il quale ci servirà per poter calcolare la differenza tra ammontare prestato e preso in prestito per ogni paese. Effettuerò quindi una divisione in due dataset, uno in cui non vi siano valori mancanti in "country_code" e uno in cui vi siano solo valori mancanti in "country_code". Estrapolata la distribuzione di "country_code" dal dataset non contenenete valori mancanti, si andrà a sostituire i valori mancanti in "country_code" nel secondo df attraverso la distribuzione trovata. In questo modo si potrà tornare al df di partenza con zero valori mancanti in "country_name" . 

Estrapolo il df contenente solo valori mancanti (in "country_code") attraverso la funzione isnull(). Utilezzerò df.loc per poter estrarre solo le osservazioni il cui attributo "country_name" presenta valori mancanti : 

In [25]:
country_code_unknow = lenders.loc[lenders["country_code"].isnull()].reset_index()
country_code_unknow.head(3)

,index,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0


Allo stesso modo, estrapolo il df non contenente valori mancanti (in "country_code") attraverso la funzione notnull() :

In [26]:
country_code_know = lenders.loc[lenders["country_code"].notnull()].reset_index()
country_code_know.head(3)

,index,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,16,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0
1,31,christina27976796,Christina,NaN,NaN,US,1461303341,NaN,NaN,1.0,Peter Tan,0
2,37,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.0,NaN,0


In [27]:
# controllo che l'operazione di splittagio del dataset originale sia andata a buon fine'

print(len(lenders) - len(country_code_know) - len(country_code_unknow))

0


Calcolo la lunghezza, ovvero il numero di osservazioni presenti nel df "country_code_know", in modo tale da poterla utilizzare nel calcolo della distribuzione dello stesso dataset :

In [28]:
tot_users_know = len(country_code_know.index)
tot_users_know

890539

Trovo la distribuzione del df "country_code_know", calcolando le frequenze relative per ogni paese : 

In [29]:
country_distr_know = country_code_know[["index", "country_code"]].groupby("country_code").count().reset_index().rename(columns = {"index":"tot_users"})
country_distr_know["%"] = country_distr_know["tot_users"]/tot_users_know*100
country_distr_know.head(3)

,country_code,tot_users,%
0,AD,15,0.001684
1,AE,1043,0.117120
2,AF,228,0.025602


Si ricorda che la somma di tutte le probabilità / freq_relative deve essere uguale 1. Per sicurezza, imposterò questa condizione :

In [30]:
country_distr_know["%"] /= country_distr_know["%"].sum()

Attraverso la funzione np.random.choice si va a sostituire in modo "randomico" (tenendo conto della distribuzione di di "country_code" del df "country_code_know") i valori mancanti di "country_code" nel df "country_code_unknow" :

In [31]:
np.random.seed(1234)
country_code_unknow["country_code"] = np.random.choice(country_distr_know["country_code"], size=len(country_code_unknow.index), p = country_distr_know["%"])
country_code_unknow.head(3)

,index,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,0,qian3013,Qian,NaN,NaN,GB,1461300457,NaN,NaN,1.0,NaN,0
1,1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0
2,2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0


Ora posso effettuare una merge per ritornare al dataset originale :

In [32]:
lenders = pd.concat([country_code_know, country_code_unknow]).drop(columns="index")
lenders.head(3)

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0
1,christina27976796,Christina,NaN,NaN,US,1461303341,NaN,NaN,1.0,Peter Tan,0
2,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.0,NaN,0


Calcolo il totale prestato da ogni singolo paese effettuando una merge tra il dataset "tot_lent_by_lender" (contiene le informazioni di quanto prestato da ogni singolo finanziatore, "lenders") e il dataset "lenders" (contiene le informazioni di ogni finanziatore) : 

In [33]:
total_lent_from_the_country = pd.merge(tot_lent_by_lender, lenders[["permanent_name", "country_code"]], left_on="lenders", right_on="permanent_name").drop(columns="permanent_name").groupby("country_code")["amount_for_lender"].sum().to_frame().rename(columns={'amount_for_lender' : 'total_lent_from_the_country'}).reset_index()
total_lent_from_the_country.head(3)

,country_code,total_lent_from_the_country
0,AD,6.412811e+03
1,AE,1.569657e+06
2,AF,1.589440e+05


Calcolo l'ammontare preso in prestito da ogni paese effettuando una groupby su "country_code" del dataset "loans_cleaned" (contiene le informazioni sui prestiti concessi ad ogni paese) e applicando una sum() all'attributo "loan_amount" :

In [34]:
total_borrowed_from_the_country =  loans_cleaned.groupby('country_code', as_index=False)["loan_amount"].sum().rename(columns={"loan_amount" : "total_borrowed_from_the_country"})
total_borrowed_from_the_country.head(3)

,country_code,total_borrowed_from_the_country
0,AF,6000.0
1,AL,4307350.0
2,AM,20579275.0


Inserisco le informazioni ottenuto in un unico dataset: 

In [35]:
summary_of_loans_by_country = pd.merge(total_lent_from_the_country, total_borrowed_from_the_country, on = "country_code")
summary_of_loans_by_country.head(3)

,country_code,total_lent_from_the_country,total_borrowed_from_the_country
0,AF,158943.967927,6000.0
1,AL,20913.605196,4307350.0
2,AM,33428.316161,20579275.0


Calcolo la differenza tra ammontare prestato e preso a prestito per ogni paese ("net_loans") :

In [36]:
summary_of_loans_by_country["net_loans"] = summary_of_loans_by_country["total_lent_from_the_country"] - summary_of_loans_by_country["total_borrowed_from_the_country"]
summary_of_loans_by_country.head(3)

,country_code,total_lent_from_the_country,total_borrowed_from_the_country,net_loans
0,AF,158943.967927,6000.0,1.529440e+05
1,AL,20913.605196,4307350.0,-4.286436e+06
2,AM,33428.316161,20579275.0,-2.054585e+07


#### 10) Which country has the highest ratio between the difference computed at the previous point and the population?

Carico il dataset "country_stats" contentente le infromazioni sulla popolazione di ogni paese :

In [37]:
country_stat = pd.read_csv('D:/UNIVERSITÀ/PROGETTI/FOCS/DATASET/country_stats.csv')
country_stat.head(3)

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico


Unisco le informazioni sulla popolazione del singolo paese del dataset "country_stat" con le informazioni sui presti (erogati e ottenuti) per ogni paese del dataset "summary_of_loans_by_country" : 

In [38]:
country_loan_stat = pd.merge(country_stat [['country_code', 'population']], summary_of_loans_by_country, on ='country_code')
country_loan_stat.head(3)

,country_code,population,total_lent_from_the_country,total_borrowed_from_the_country,net_loans
0,IN,1339180127,2.012132e+06,8676050.0,-6.663918e+06
1,NG,190886311,2.013368e+05,1542950.0,-1.341613e+06
2,MX,129163276,1.340695e+06,15656225.0,-1.431553e+07


Calcolo il rapporto tra i prestiti netti ("net_loans") e popolazione ("population) per pgni singolo paese :

In [39]:
country_loan_stat["net_loans/pop"] = country_loan_stat["net_loans"]/country_loan_stat["population"]
country_loan_stat.head(3)

,country_code,population,total_lent_from_the_country,total_borrowed_from_the_country,net_loans,net_loans/pop
0,IN,1339180127,2.012132e+06,8676050.0,-6.663918e+06,-0.004976
1,NG,190886311,2.013368e+05,1542950.0,-1.341613e+06,-0.007028
2,MX,129163276,1.340695e+06,15656225.0,-1.431553e+07,-0.110833


Estraggo il paese con il rapporto "net_loans/pop" più alto :

In [40]:
country_loan_stat[country_loan_stat['net_loans/pop'] == country_loan_stat['net_loans/pop'].max()]

,country_code,population,total_lent_from_the_country,total_borrowed_from_the_country,net_loans,net_loans/pop
5,US,324459463,5.188974e+08,27809375.0,4.910881e+08,1.513557


#### 11)  Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

Prefersico tener conto solo delle osservazioni con un effettivo valore nell'attributo "population_below_poverty_line" :

In [41]:
country_stat_know = country_stat[country_stat["population_below_poverty_line"].notnull() == True]

Calcolo la popolazione al di sopra della soglia di povertà

In [42]:
country_stat_know["population_above_poverty_line"] = (country_stat_know["population"] -(country_stat_know["population"]*country_stat_know["population_below_poverty_line"]/100))
country_stat_know.head(3)

<ipython-input-42-c965275f7f00>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_stat_know["population_above_poverty_line"] = (country_stat_know["population"] -(country_stat_know["population"]*country_stat_know["population_below_poverty_line"]/100))


,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,population_above_poverty_line
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,1.045900e+09
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,5.726589e+07
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,6.948984e+07


Aggiungo la nuova informazione ottenuta, ovvero l'attributo "population_above_poverty_line", al dataset "country_loan_stat" :

In [43]:
country_loan_stat  = pd.merge(country_loan_stat, country_stat_know[["country_code", "population_above_poverty_line"]], on="country_code")
country_loan_stat.head(3)

,country_code,population,total_lent_from_the_country,total_borrowed_from_the_country,net_loans,net_loans/pop,population_above_poverty_line
0,IN,1339180127,2.012132e+06,8676050.0,-6.663918e+06,-0.004976,1.045900e+09
1,NG,190886311,2.013368e+05,1542950.0,-1.341613e+06,-0.007028,5.726589e+07
2,MX,129163276,1.340695e+06,15656225.0,-1.431553e+07,-0.110833,6.948984e+07


Calcolo il rapporto tra net_loans e population_above_poverty_line :

In [44]:
country_loan_stat['netloans_population_above_poverty_ratio'] = country_loan_stat["net_loans"]/country_loan_stat["population_above_poverty_line"]
country_loan_stat.head(3)

,country_code,population,total_lent_from_the_country,total_borrowed_from_the_country,net_loans,net_loans/pop,population_above_poverty_line,netloans_population_above_poverty_ratio
0,IN,1339180127,2.012132e+06,8676050.0,-6.663918e+06,-0.004976,1.045900e+09,-0.006371
1,NG,190886311,2.013368e+05,1542950.0,-1.341613e+06,-0.007028,5.726589e+07,-0.023428
2,MX,129163276,1.340695e+06,15656225.0,-1.431553e+07,-0.110833,6.948984e+07,-0.206009


Trovo lo stato con il rapporto net_loans e population_above_poverty_line maggiore : 

In [45]:
country_loan_stat[country_loan_stat['netloans_population_above_poverty_ratio']==country_loan_stat['netloans_population_above_poverty_ratio'].max()]

,country_code,population,total_lent_from_the_country,total_borrowed_from_the_country,net_loans,net_loans/pop,population_above_poverty_line,netloans_population_above_poverty_ratio
5,US,324459463,5.188974e+08,27809375.0,4.910881e+08,1.513557,2.754661e+08,1.782753


#### 12)  For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different 

#### years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time 

#### December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / 
#### (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Inizio ripulendo il dataset loans_cleaned da eventuali valori Nan negli attributi "disburse_time" e "planned_expiration_time" : 

In [46]:
loans_cleaned=loans_cleaned.dropna(subset=['planned_expiration_time','disburse_time'])

In [47]:
print(len(loans_cleaned))

1029468


Creo il dataset contenente solo i loans in cui "disburse_time" e "planned_expiration_time" hanno lo stesso anno :

In [48]:
same_year = loans_cleaned[loans_cleaned['disburse_time'].dt.year == loans_cleaned['planned_expiration_time'].dt.year][['loan_id', "disburse_time", 'planned_expiration_time', 'loan_amount']].reset_index(drop=True)
same_year.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,658010,2014-01-09,2014-02-15,400.0
1,659347,2014-01-17,2014-02-21,625.0
2,659605,2014-01-15,2014-02-20,350.0
3,660240,2014-01-20,2014-02-21,125.0
4,661601,2014-01-10,2014-02-25,1600.0


In [49]:
len(same_year)

878857

Calcolo l'ammontare dei prestiti per ogni anno (sul dataset same_year) : 

In [50]:
loans_amount_same_year = same_year.groupby(same_year['disburse_time'].dt.year)['loan_amount'].sum().to_frame().reset_index()
loans_amount_same_year = loans_amount_same_year.rename(columns= {"disburse_time" : "year", "loan_amount" : "loans_amount_same_year"})
loans_amount_same_year

,year,loans_amount_same_year
0,2012,103880625.0
1,2013,98372250.0
2,2014,120456375.0
3,2015,130943675.0
4,2016,132997175.0
5,2017,144634400.0
6,2018,85300.0


Creo il dataset contenente solo i loans in cui "disburse_time" e "planned_expiration_time" hanno anni diversi :

In [51]:
different_year = loans_cleaned[loans_cleaned['disburse_time'].dt.year != loans_cleaned['planned_expiration_time'].dt.year][['loan_id', "disburse_time", 'planned_expiration_time', 'loan_amount']]
len(different_year)

150611

Confronto le lunghezze dei tre dataset come controllo :

In [52]:
print(len(loans_cleaned) - len(same_year) - len(different_year))

0


Definisco la funzione con cui calcolare per ogni anno del prestito l'ammontare da computare ad ogni anno. 
La funzione richiede una certa capacità computazionale e di memoria (RAM), proprio per questo motivo ho diviso il set di dati in due. In realtà questa funzione porta al risultato finale anche con l'intero dataset iniziale (loans_cleaned).

In [53]:
def func(disburse_time, planned_time, loan_amount):
    total_cost=loan_amount
    dict_map = {}
    for year in range(disburse_time.year, planned_time.year+1):
        if year==disburse_time.year:
            dict_map[year] = (pd.to_datetime(datetime.date(year, 12, 31)) - disburse_time).days+1
        elif year==planned_time.year:
            dict_map[year] = (planned_time - pd.to_datetime(datetime.date(year-1, 12, 31))).days
        else:
            if year%4==0:
                dict_map[year]=366
            else:
                dict_map[year]=365
    
    
    dict_year_share = {year:total_cost*days/sum(dict_map.values()) for year,days in dict_map.items()}
    return dict_year_share

Applico ora la funzione al dataset different_year :

In [54]:
result = different_year.apply(lambda x: pd.Series(func(x['disburse_time'], x['planned_expiration_time'],x['loan_amount'])), axis=1)

In [55]:
result = result.reset_index(drop=True)
result.head()

,2011,2012,2013,2014,2015,2016,2017,2018
0,NaN,NaN,22.727273,102.272727,NaN,NaN,NaN,NaN
1,NaN,NaN,49.484536,350.515464,NaN,NaN,NaN,NaN
2,NaN,NaN,108.050847,316.949153,NaN,NaN,NaN,NaN
3,NaN,NaN,173.360656,1001.639344,NaN,NaN,NaN,NaN
4,NaN,NaN,19.780220,280.219780,NaN,NaN,NaN,NaN


In [56]:
# confronto le lunghezze dei tre dataset come controllo

print(len(loans_cleaned) - len(same_year) - len(result))

0


Estraggo l'insieme degli anni così da poter creare un nuovo df dove :
- nella colonna "year", sono riportati tutti gli anni coinvolti nei prestiti; 
- nella colonna 'loans_amount_different_year' ,  il valore totale dei presti per ogni singolo anno :

In [57]:
max_year=different_year[["disburse_time","planned_expiration_time"]].max().max().year
min_year=different_year[["disburse_time","planned_expiration_time"]].min().min().year

years=[str(year) for year in range(min_year,max_year+1)]
years

['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [58]:
loans_amount_different_year = pd.DataFrame()
diff_summary= pd.DataFrame(result.sum())

In [59]:
loans_amount_different_year['year'] = diff_summary.index
loans_amount_different_year['loans_amount_different_year'] = diff_summary.values
loans_amount_different_year

,year,loans_amount_different_year
0,2011,6.614928e+05
1,2012,9.251024e+06
2,2013,2.512557e+07
3,2014,3.001850e+07
4,2015,2.394231e+07
5,2016,2.081552e+07
6,2017,2.046922e+07
7,2018,4.877086e+06


Unisco tutto in un unico dataset :

In [60]:
loans_amount_for_year = pd.merge(loans_amount_different_year, loans_amount_same_year, on = 'year', how='outer')
loans_amount_for_year

,year,loans_amount_different_year,loans_amount_same_year
0,2011,6.614928e+05,NaN
1,2012,9.251024e+06,103880625.0
2,2013,2.512557e+07,98372250.0
3,2014,3.001850e+07,120456375.0
4,2015,2.394231e+07,130943675.0
5,2016,2.081552e+07,132997175.0
6,2017,2.046922e+07,144634400.0
7,2018,4.877086e+06,85300.0


In [61]:
loans_amount_for_year = loans_amount_for_year.fillna(0)

In [62]:
loans_amount_for_year['tot_loans'] = loans_amount_for_year['loans_amount_different_year'] + loans_amount_for_year['loans_amount_same_year']
loans_amount_for_year

,year,loans_amount_different_year,loans_amount_same_year,tot_loans
0,2011,6.614928e+05,0.0,6.614928e+05
1,2012,9.251024e+06,103880625.0,1.131316e+08
2,2013,2.512557e+07,98372250.0,1.234978e+08
3,2014,3.001850e+07,120456375.0,1.504749e+08
4,2015,2.394231e+07,130943675.0,1.548860e+08
5,2016,2.081552e+07,132997175.0,1.538127e+08
6,2017,2.046922e+07,144634400.0,1.651036e+08
7,2018,4.877086e+06,85300.0,4.962386e+06


In [63]:
print(loans_amount_for_year['tot_loans'].sum() - tot_loans)

-2.384185791015625e-07


La differenza, trascurabilissima,  è dovuta a errori in fase di approssimizzazione nel calcolo dell'ammontare da computare di ogni prestito per ogni anno del prestito.